## PROJECT GOAL:
Outbrain is a web advertising platform that displays advertisement boxes of links to pages within websites. It displays links to the sites' pages in addition to sponsored content, generating revenue from the latter.
In our project we are predicting whether the particular ad_id will be clicked or not.

This python files predict ad using random forest classifier algorithm and measure Accuracy and AUC.

## Creating a Classification Model using Gradient Boosting Algorithm

In this file, we will implement a classification model using *Gradient Boosting that uses features of a Event,Document and Train dataframe to **predict the whether the ad is clicked or not** 

You should follow the steps below to build, train and test the model from the source data:

1. Build a schema of a source data for its Data Frame
2. Load the Source Data to the schema
3. Prepare the data with the features (input columns, output column as label)
4. Split the data using data.randomSplit(): Training and Testing
5. Transform the columns to a vector using VectorAssembler
6. set features and label from the vector
7. Build a ***Gradient Boosting Algorithm*** Model with the label and features
8. Train the model
9. Prepare the testing Data Frame with features and label from the vector; Rename label to trueLabel
10. Predict and test the testing Data Frame using the model trained at the step 8
11. Compare the predicted result and trueLabel


### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [3]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.mllib.tree import RandomForest
from pyspark.ml import Pipeline

from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier


#####The below command will start spark session when we run our file in oracle BDE. In Databricks keep this cell as False by default. But when you run file in Oracle BDE make it True.

In [5]:
#This 
IS_SPARK_SUBMIT_CLI = False
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

### Load Source Data
The data for this project is provided as a CSV file containing details of advertisement. The data includes specific characteristics (or *features*) for each ad, as well as a *label* column indicating whether the ad was clicked or not.

You will load this data into a DataFrame and display it.

#####Reading all necessary csv file

In [8]:
if IS_SPARK_SUBMIT_CLI:
    event = spark.read.csv('eventd.csv', inferSchema=True, header=True)
else:
    event = spark.sql("SELECT * FROM eventd_csv")

In [9]:
if IS_SPARK_SUBMIT_CLI:
    train = spark.read.csv('traind.csv', inferSchema=True, header=True)
else:
    train = spark.sql("SELECT * FROM traind_csv")

In [10]:
if IS_SPARK_SUBMIT_CLI:
    promoted_content = spark.read.csv('promoted_content.csv', inferSchema=True, header=True)
else:
    promoted_content = spark.sql("SELECT * FROM promoted_content_csv")

Below command will join event and train file using disply_id as primary key. It will also drop display_id from event table and ad_id from train table as it create ambiguity error. Then it will assign result to new dataframe data1.

In [12]:
data1=event.join(train,event.display_id==train.display_id).drop(event.display_id).drop(train.ad_id)

Following cell will join two dataframe data1 (which created from joining of event and train table) with Promoted content dataframe using document_id as primary key. Then it will assign result to new dataframe data2.

In [14]:
data2=data1.join(promoted_content,data1.document_id==promoted_content.document_id).drop(data1.document_id)

Following table select disply_id,document_id,platform,ad_id,camaign_id,advertiser_id which are features which help us to train our dataset and clicked column which is label column. Selected column will assign to new dataframe called data.

In [16]:
data = data2.select("display_id", "document_id", "platform", "ad_id", "campaign_id","advertiser_id",col("clicked").alias("label"))

### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [18]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

### Prepare the Training Data create Pipeline
To train the classification model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **VectorAssembler** class to transform the feature columns into a vector,Normlize data using Minmax.

In [20]:
from pyspark.ml.classification import RandomForestClassifier

assembler = VectorAssembler(inputCols = ["display_id", "document_id", "platform", "ad_id","campaign_id","advertiser_id"],outputCol="normfeatures")
#assembler = VectorAssembler(inputCols = ["clicked"],outputCol="label")
minMax = MinMaxScaler(inputCol = assembler.getOutputCol(), outputCol="nfeatures")
featVect = VectorAssembler(inputCols=["nfeatures"], outputCol="features")
#rf = RandomForestClassifier(labelCol="label", featuresCol="features",impurity="gini",featureSubsetStrategy="auto",numTrees=10,maxDepth=32,maxBins=128,seed=1234)
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)
pipeline = Pipeline(stages=[assembler,minMax,featVect,gbt])

The pipeline itself is an estimator, and so it has a fit method that you can call to run the pipeline on a specified DataFrame. In this case, you will run the pipeline on the training data to train a model.

In [22]:
piplineModel = pipeline.fit(train)
print("Pipeline complete!")

Pipeline complete!

### Test the Recommender
The model produced by the pipeline is a transformed that will apply to all stages in the pipeline to a specified DataFrame and apply the trained model to generate predictions. In this case, you will transform the test DataFrame using the pipeline to generate label prediction

In [24]:
prediction = piplineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+---------------------------------------------------------------------------------------------------------+----------+---------+
features |prediction|trueLabel|
+---------------------------------------------------------------------------------------------------------+----------+---------+
[0.23366817877427618,0.5000774396069431,0.0,0.28502742329432706,0.5789429127747834,0.38717002554640934] |0.0 |1 |
[0.23366817877427618,0.5000774396069431,0.0,0.31694054993452325,0.5789429127747834,0.38717002554640934] |0.0 |1 |
[0.23366817877427618,0.5000774396069431,0.0,0.31694579506573023,0.5789429127747834,0.38717002554640934] |0.0 |1 |
[0.23366817877427618,0.5000774396069431,0.0,0.3169510401969372,0.5789429127747834,0.38717002554640934] |0.0 |1 |
[0.5977541407836479,0.7992039279944,0.0,0.8050751889558517,0.020966786127493864,0.002554640931024695] |0.0 |0 |
[0.7016187812154383,0.756300954993102,1.0,0.7834355259730156,0.34760278804639216,0.08174850979279023] |0.0 |0 |
[0.10106117287015641,0.27458689636005235,0.5,0.07751954248468859,0.05909078082230437,0.01107011070110701]|0.0 |0 |
[0.10106117287015641,0.27458689636005235,0.5,0.07752828437003352,0.05909078082230437,0.01107011070110701]|0.0 |0 |
[0.2029942614113061,0.7174277030625665,0.0,0.7099267604845801,0.006885458701357338,2.838489923360772E-4] |0.0 |0 |
[0.524338853152041,0.27458689636005235,0.0,0.07752478761589554,0.05909078082230437,0.01107011070110701] |0.0 |1 |
[0.524338853152041,0.27458689636005235,0.0,0.07842520180642319,0.001890679233569433,0.01107011070110701] |0.0 |1 |
[0.5587251757521771,0.27458689636005235,1.0,0.07752303923882656,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.5587251757521771,0.27458689636005235,1.0,0.07752478761589554,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.6679213408183441,0.8870200845790737,0.5,0.8839497236690043,0.02342184722183029,0.04598353675844451] |0.0 |0 |
[0.9036495805567077,0.27458689636005235,1.0,0.07752303923882656,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9036495805567077,0.27458689636005235,1.0,0.07752653599296454,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9036495805567077,0.27458689636005235,1.0,0.07752828437003352,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9036495805567077,0.27458689636005235,1.0,0.07842520180642319,0.001890679233569433,0.01107011070110701]|0.0 |0 |
[0.9450505050744769,0.27458689636005235,1.0,0.07751779410761961,0.14343764991393176,0.01107011070110701] |0.0 |0 |
[0.9450505050744769,0.27458689636005235,1.0,0.07751954248468859,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9450505050744769,0.27458689636005235,1.0,0.07752828437003352,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.07619282792403678,0.6488977643382204,1.0,0.6281411779515664,0.006885458701357338,2.838489923360772E-4]|0.0 |0 |
[0.04533795364785702,0.5855832847770902,1.0,0.47103900804078613,0.7301690323672997,0.027817201248935566] |0.0 |0 |
[0.04533795364785702,0.5855832847770902,1.0,0.49839236728506764,0.7301408132742614,0.027817201248935566] |0.0 |0 |
[0.05260132682355176,0.5855832847770902,0.0,0.49839236728506764,0.7301408132742614,0.027817201248935566] |0.0 |1 |
[0.31112936633457416,0.5855832847770902,0.0,0.47103900804078613,0.7301690323672997,0.027817201248935566] |0.0 |0 |
[0.31112936633457416,0.5855832847770902,0.0,0.49839236728506764,0.7301408132742614,0.027817201248935566] |0.0 |0 |
[0.3490723525662449,0.767665440867438,0.5,0.7915130280317295,0.02342184722183029,0.04598353675844451] |0.0 |0 |
[0.5054804729394955,0.5855832847770902,1.0,0.4769834900753376,0.7301408132742614,0.027817201248935566] |0.0 |0 |
[0.5933344610014417,0.5855832847770902,1.0,0.4769834900753376,0.7301408132742614,0.027817201248935566] |0.0 |0 |
[0.7094492911561728,0.5855832847770902,0.0,0.49839236728506764,0.7301408132742614,0.027817201248935566] |0.0 |0 |
[0.7565316101172358,0.5855832847770902,1.0,0.4769834900753376,0.7301408132742614,0.027817201248935566] |0.0 |0 |
[0.7565316101172358,0.5855832847770902,1.0,0.498392367

Following cell measure accuracy of Algorithm using MultiClassificationEvaluater and evaluate using predicted data.

In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluation = MulticlassClassificationEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluation.evaluate(prediction)
print(accuracy)

0.8984530229917683

Following Code will give total Error in project

In [28]:
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.101547

#### Gradient Boosting Evaluator
Calculate AUC using Gradient Boosting Evaluator.

In [30]:
rf_evaluator =  MulticlassClassificationEvaluator(labelCol="trueLabel", predictionCol="prediction")
rf_auc = rf_evaluator.evaluate(prediction)
print("AUC for Gradient Boost is= ", rf_auc)

AUC for Gradient Boost is= 0.8750931883418767

#### Gradient Boosting Confusion matrix
Calculate confusion matrix and measure precision,Recall

In [32]:
# Only for Classification Logistic Regression not for Linear Regression

tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])
metrics.show()

+---------+-------------------+
 metric| value|
+---------+-------------------+
 TP| 653.0|
 FP| 10.0|
 TN| 12008.0|
 FN| 1421.0|
Precision| 0.9849170437405732|
 Recall|0.31485053037608485|
+---------+-------------------+